Load required libraries. int_nmf_model must be in the same directory. If it is not it can be added to pythons path

In [ ]:
import anndata as ad
import scanpy as sc
import numpy as np
import scipy
import sys
import seaborn as sns
sys.path.append("../../CellPie/") # go to parent dir
import cellpie_main as cp
from cellpie_main import intNMF
import anndata as ad
from matplotlib import pyplot as plt
import scipy.io
import pandas as pd
import squidpy as sq
from PIL import Image

In [ ]:
sc.set_figure_params(scanpy=True, dpi=250, dpi_save=150)

In [ ]:
# read ST data
adata = sc.read_visium(path = 'Data/', 
                       count_file='filtered_feature_bc_matrix.h5', 
                       library_id='A1_spot', 
                       load_images=True)
adata.var_names_make_unique()
adata.var['SYMBOL'] = adata.var_names
sc.pp.calculate_qc_metrics(adata, inplace=True)
adata.var['mt'] = [gene.startswith('mt-') for gene in adata.var['SYMBOL']]
adata.obs['mt_frac'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']

In [ ]:
sc.pp.filter_genes(adata, min_cells=30)

In [ ]:
sc.pp.normalize_total(adata)

In [ ]:
img_n = sq.im.ImageContainer(
adata.uns["spatial"]['A1_spot']["images"]["hires"][:,:,0],
scale=adata.uns["spatial"]['A1_spot']["scalefactors"]["tissue_hires_scalef"])

In [ ]:
img_g = sq.im.ImageContainer(
adata.uns["spatial"]['A1_spot']["images"]["hires"][:,:,2],
scale=adata.uns["spatial"]['A1_spot']["scalefactors"]["tissue_hires_scalef"])

In [ ]:
img = sq.im.ImageContainer(
adata.uns["spatial"]['A1_spot']["images"]["hires"],
scale=adata.uns["spatial"]['A1_spot']["scalefactors"]["tissue_hires_scalef"])

In [ ]:
# cp.extract_image_features(adata,scale_img=1,mask_circle = False,
#     spot_scale = 2.5)

In [ ]:
sq.im.calculate_image_features(
    adata,
    img,
    features="histogram",
    features_kwargs={"histogram": {"bins": 10}},
    mask_circle = False,
    spot_scale = 2.5,
    key_added="features"
)

In [ ]:
adata_g = adata.copy()
adata_n = adata.copy()

## extract intensity values from neurons and glia image channels - ground truth

In [ ]:
sq.im.calculate_image_features(
    adata_g,
    img_g,
    features="summary",
    mask_circle=True,
    show_progress_bar=True,
    scale=1,
    key_added="features"
)

In [ ]:
sq.im.calculate_image_features(
    adata_n,
    img_n,
    features="summary",
    mask_circle=True,
    show_progress_bar=True,
    scale=1,
    key_added="features"
)

In [ ]:
adata_g.obs['glial'] = adata_g.obsm['features']['summary_ch-0_mean']
adata_n.obs['neurons'] = adata_n.obsm['features']['summary_ch-0_mean']

In [ ]:
# int_nmf_model_update.model_selection(adata,3,60)

In [ ]:
nmf_model = intNMF(adata,17, epochs = 60, init = 'NNDSVD',mod1_skew=1,random_state=2)
nmf_model.fit(adata)

In [ ]:
cp.plot_topic_proportions(adata,17)

In [ ]:
# this function has been taken from https://github.com/BayraktarLab/cell2location

import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec


def get_rgb_function(cmap, min_value, max_value):
    r"""Generate a function to map continous values to RGB values using colormap between min_value & max_value."""

    if min_value > max_value:
        raise ValueError("Max_value should be greater or than min_value.")

    if min_value == max_value:
        warnings.warn(
            "Max_color is equal to min_color. It might be because of the data or bad parameter choice. "
            "If you are using plot_contours function try increasing max_color_quantile parameter and"
            "removing cell types with all zero values."
        )

        def func_equal(x):
            factor = 0 if max_value == 0 else 0.5
            return cmap(np.ones_like(x) * factor)

        return func_equal

    def func(x):
        return cmap((np.clip(x, min_value, max_value) - min_value) / (max_value - min_value))

    return func


def rgb_to_ryb(rgb):
    """
    Converts colours from RGB colorspace to RYB
    Parameters
    ----------
    rgb
        numpy array Nx3
    Returns
    -------
    Numpy array Nx3
    """
    rgb = np.array(rgb)
    if len(rgb.shape) == 1:
        rgb = rgb[np.newaxis, :]

    white = rgb.min(axis=1)
    black = (1 - rgb).min(axis=1)
    rgb = rgb - white[:, np.newaxis]

    yellow = rgb[:, :2].min(axis=1)
    ryb = np.zeros_like(rgb)
    ryb[:, 0] = rgb[:, 0] - yellow
    ryb[:, 1] = (yellow + rgb[:, 1]) / 2
    ryb[:, 2] = (rgb[:, 2] + rgb[:, 1] - yellow) / 2

    mask = ~(ryb == 0).all(axis=1)
    if mask.any():
        norm = ryb[mask].max(axis=1) / rgb[mask].max(axis=1)
        ryb[mask] = ryb[mask] / norm[:, np.newaxis]

    return ryb + black[:, np.newaxis]


def ryb_to_rgb(ryb):
    """
    Converts colours from RYB colorspace to RGB
    Parameters
    ----------
    ryb
        numpy array Nx3
    Returns
    -------
    Numpy array Nx3
    """
    ryb = np.array(ryb)
    if len(ryb.shape) == 1:
        ryb = ryb[np.newaxis, :]

    black = ryb.min(axis=1)
    white = (1 - ryb).min(axis=1)
    ryb = ryb - black[:, np.newaxis]

    green = ryb[:, 1:].min(axis=1)
    rgb = np.zeros_like(ryb)
    rgb[:, 0] = ryb[:, 0] + ryb[:, 1] - green
    rgb[:, 1] = green + ryb[:, 1]
    rgb[:, 2] = (ryb[:, 2] - green) * 2

    mask = ~(ryb == 0).all(axis=1)
    if mask.any():
        norm = rgb[mask].max(axis=1) / ryb[mask].max(axis=1)
        rgb[mask] = rgb[mask] / norm[:, np.newaxis]

    return rgb + white[:, np.newaxis]


def plot_spatial_general(
    value_df,
    coords,
    labels,
    text=None,
    circle_diameter=4.0,
    alpha_scaling=1.0,
    max_col=(np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf,np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf,np.inf,np.inf,np.inf),
    max_color_quantile=0.98,
    show_img=True,
    img=None,
    img_alpha=1.0,
    adjust_text=False,
    plt_axis="off",
    axis_y_flipped=True,
    x_y_labels=("", ""),
    crop_x=None,
    crop_y=None,
    text_box_alpha=0.9,
    reorder_cmap=range(17),
    style="fast",
    colorbar_position="bottom",
    colorbar_label_kw={},
    colorbar_shape={},
    colorbar_tick_size=12,
    colorbar_grid=None,
    image_cmap="Greys_r",
    white_spacing=20,
):
    r"""Plot spatial abundance of cell types (regulatory programmes) with colour gradient and interpolation.
      This method supports only 7 cell types with these colours (in order, which can be changed using reorder_cmap).
      'yellow' 'orange' 'blue' 'green' 'purple' 'grey' 'white'
    :param value_df: pd.DataFrame - with cell abundance or other features (only 7 allowed, columns) across locations (rows)
    :param coords: np.ndarray - x and y coordinates (in columns) to be used for ploting spots
    :param text: pd.DataFrame - with x, y coordinates, text to be printed
    :param circle_diameter: diameter of circles
    :param labels: list of strings, labels of cell types
    :param alpha_scaling: adjust color alpha
    :param max_col: crops the colorscale maximum value for each column in value_df.
    :param max_color_quantile: crops the colorscale at x quantile of the data.
    :param show_img: show image?
    :param img: numpy array representing a tissue image.
        If not provided a black background image is used.
    :param img_alpha: transparency of the image
    :param lim: x and y max limits on the plot. Minimum is always set to 0, if `lim` is None maximum
        is set to image height and width. If 'no_limit' then no limit is set.
    :param adjust_text: move text label to prevent overlap
    :param plt_axis: show axes?
    :param axis_y_flipped: flip y axis to match coordinates of the plotted image
    :param reorder_cmap: reorder colors to make sure you get the right color for each category
    :param style: plot style (matplolib.style.context):
        'fast' - white background & dark text;
        'dark_background' - black background & white text;
    :param colorbar_position: 'bottom', 'right' or None
    :param colorbar_label_kw: dict that will be forwarded to ax.set_label()
    :param colorbar_shape: dict {'vertical_gaps': 1.5, 'horizontal_gaps': 1.5,
                                    'width': 0.2, 'height': 0.2}, not obligatory to contain all params
    :param colorbar_tick_size: colorbar ticks label size
    :param colorbar_grid: tuple of colorbar grid (rows, columns)
    :param image_cmap: matplotlib colormap for grayscale image
    :param white_spacing: percent of colorbars to be hidden
    """

    if value_df.shape[1] > 17:
        raise ValueError("Maximum of 7 cell types / factors can be plotted at the moment")

    def create_colormap(R, G, B):
        spacing = int(white_spacing * 2.55)

        N = 255
        M = 3

        alphas = np.concatenate([[0] * spacing * M, np.linspace(0, 1.0, (N - spacing) * M)])

        vals = np.ones((N * M, 4))
        #         vals[:, 0] = np.linspace(1, R / 255, N * M)
        #         vals[:, 1] = np.linspace(1, G / 255, N * M)
        #         vals[:, 2] = np.linspace(1, B / 255, N * M)
        for i, color in enumerate([R, G, B]):
            vals[:, i] = color / 255
        vals[:, 3] = alphas

        return ListedColormap(vals)
    cmaps = []
#     # Create linearly scaled colormaps
#     YellowCM = create_colormap(240, 228, 66)  # #F0E442 ['#F0E442', '#D55E00', '#56B4E9',
#     # '#009E73', '#5A14A5', '#C8C8C8', '#323232']
    RedCM = create_colormap(199, 49, 38)  # #D55E00
    BlueCM = create_colormap(86, 180, 233)  # #56B4E9
    GreenCM = create_colormap(0,100,0)  # #009E73
#     GreyCM = create_colormap(200, 200, 200)  # #C8C8C8
#     WhiteCM = create_colormap(50, 50, 50)  # #323232
#     PurpleCM = create_colormap(90, 20, 165)  # #5A14A5
    
    Bl1=create_colormap(186, 71, 67)
    Bl2=create_colormap(73, 99, 82)
    Bl3=create_colormap(173, 107, 57)
    Bl4=create_colormap(125, 53, 133)
    Bl5=create_colormap(110, 117, 133)
    Bl6=create_colormap(35, 105, 110)
    Bl7=create_colormap(184, 136, 186)
    Bl8=create_colormap(217, 194, 189)
    Bl9=create_colormap(162, 196, 198)
    Bl10=create_colormap(65, 87, 135)
    Bl11=create_colormap(124, 137, 126)
    Bl12=create_colormap(219, 212, 66)
    Bl13=create_colormap(64, 57, 62)
    Bl14=create_colormap(174, 210, 140)
    Bl15=create_colormap(152, 89, 171)
    Bl16=create_colormap(171, 171, 89)
    Bl17=create_colormap(89, 171, 112)
    
    cmaps = [RedCM,BlueCM,GreenCM,Bl4,Bl5,Bl6,Bl7,Bl8,Bl9,Bl10,Bl11,Bl12,Bl13,Bl14,Bl15,Bl16,Bl17]

    cmaps = [cmaps[i] for i in reorder_cmap]

    with mpl.style.context(style):

        fig = plt.figure()

        if colorbar_position == "right":

            if colorbar_grid is None:
                colorbar_grid = (len(labels), 1)

            shape = {"vertical_gaps": 1.5, "horizontal_gaps": 0, "width": 0.15, "height": 0.2}
            shape = {**shape, **colorbar_shape}

            gs = GridSpec(
                nrows=colorbar_grid[0] + 2,
                ncols=colorbar_grid[1] + 1,
                width_ratios=[1, *[shape["width"]] * colorbar_grid[1]],
                height_ratios=[1, *[shape["height"]] * colorbar_grid[0], 1],
                hspace=shape["vertical_gaps"],
                wspace=shape["horizontal_gaps"],
            )
            ax = fig.add_subplot(gs[:, 0], aspect="equal", rasterized=True)

        if colorbar_position == "bottom":
            if colorbar_grid is None:
                if len(labels) <= 3:
                    colorbar_grid = (1, len(labels))
                else:
                    n_rows = round(len(labels) / 3 + 0.5 - 1e-9)
                    colorbar_grid = (n_rows, 3)

            shape = {"vertical_gaps": 0.3, "horizontal_gaps": 0.6, "width": 0.2, "height": 0.035}
            shape = {**shape, **colorbar_shape}

            gs = GridSpec(
                nrows=colorbar_grid[0] + 1,
                ncols=colorbar_grid[1] + 2,
                width_ratios=[0.3, *[shape["width"]] * colorbar_grid[1], 0.3],
                height_ratios=[1, *[shape["height"]] * colorbar_grid[0]],
                hspace=shape["vertical_gaps"],
                wspace=shape["horizontal_gaps"],
            )

            ax = fig.add_subplot(gs[0, :], aspect="equal", rasterized=True)

        if colorbar_position is None:
            ax = fig.add_subplot(aspect="equal", rasterized=True)

        if colorbar_position is not None:
            cbar_axes = []
            for row in range(1, colorbar_grid[0] + 1):
                for column in range(1, colorbar_grid[1] + 1):
                    cbar_axes.append(fig.add_subplot(gs[row, column]))

            n_excess = colorbar_grid[0] * colorbar_grid[1] - len(labels)
            if n_excess > 0:
                for i in range(1, n_excess + 1):
                    cbar_axes[-i].set_visible(False)

        ax.set_xlabel(x_y_labels[0])
        ax.set_ylabel(x_y_labels[1])

        if img is not None and show_img:
            ax.imshow(img, aspect="equal", alpha=img_alpha, origin="lower", cmap=image_cmap)

        # crop images in needed
        if crop_x is not None:
            ax.set_xlim(crop_x[0], crop_x[1])
        if crop_y is not None:
            ax.set_ylim(crop_y[0], crop_y[1])

        if axis_y_flipped:
            ax.invert_yaxis()

        if plt_axis == "off":
            for spine in ax.spines.values():
                spine.set_visible(False)
            ax.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)

        counts = value_df.values.copy()

        # plot spots as circles
        c_ord = list(np.arange(0, counts.shape[1]))

        colors = np.zeros((*counts.shape, 4))
        weights = np.zeros(counts.shape)

        for c in c_ord:

            min_color_intensity = counts[:, c].min()
            max_color_intensity = np.min([np.quantile(counts[:, c], max_color_quantile), max_col[c]])

            rgb_function = get_rgb_function(cmap=cmaps[c], min_value=min_color_intensity, max_value=max_color_intensity)

            color = rgb_function(counts[:, c])
            color[:, 3] = color[:, 3] * alpha_scaling

            norm = mpl.colors.Normalize(vmin=min_color_intensity, vmax=max_color_intensity)

            if colorbar_position is not None:
                cbar_ticks = [
                    min_color_intensity,
                    np.mean([min_color_intensity, max_color_intensity]),
                    max_color_intensity,
                ]
                cbar_ticks = np.array(cbar_ticks)

                if max_color_intensity > 13:
                    cbar_ticks = cbar_ticks.astype(np.int32)
                else:
                    cbar_ticks = cbar_ticks.round(2)

                cbar = fig.colorbar(
                    mpl.cm.ScalarMappable(norm=norm, cmap=cmaps[c]),
                    cax=cbar_axes[c],
                    orientation="horizontal",
                    extend="both",
                    ticks=cbar_ticks,
                )

                cbar.ax.tick_params(labelsize=colorbar_tick_size)
                max_color = rgb_function(max_color_intensity / 1.5)
                cbar.ax.set_title(labels[c], **{**{"size": 20, "color": max_color, "alpha": 1}, **colorbar_label_kw})

            colors[:, c] = color
            weights[:, c] = np.clip(counts[:, c] / (max_color_intensity + 1e-10), 0, 1)
            weights[:, c][counts[:, c] < min_color_intensity] = 0

        colors_ryb = np.zeros((*weights.shape, 3))

        for i in range(colors.shape[0]):
            colors_ryb[i] = rgb_to_ryb(colors[i, :, :3])

        def kernel(w):
            return w**2

        kernel_weights = kernel(weights[:, :, np.newaxis])
        weighted_colors_ryb = (colors_ryb * kernel_weights).sum(axis=1) / kernel_weights.sum(axis=1)

        weighted_colors = np.zeros((weights.shape[0], 4))

        weighted_colors[:, :3] = ryb_to_rgb(weighted_colors_ryb)

        weighted_colors[:, 3] = colors[:, :, 3].max(axis=1)

        ax.scatter(x=coords[:, 0], y=coords[:, 1], c=weighted_colors, s=circle_diameter**2)

        # add text
        if text is not None:
            bbox_props = dict(boxstyle="round", ec="0.5", alpha=text_box_alpha, fc="w")
            texts = []
            for x, y, s in zip(
                np.array(text.iloc[:, 0].values).flatten(),
                np.array(text.iloc[:, 1].values).flatten(),
                text.iloc[:, 2].tolist(),
            ):
                texts.append(ax.text(x, y, s, ha="center", va="bottom", bbox=bbox_props))

            if adjust_text:
                from adjustText import adjust_text

                adjust_text(texts, arrowprops=dict(arrowstyle="->", color="w", lw=0.5))

    return fig


def plot_spatial(adata, color, img_key="hires", show_img=True, **kwargs):
    """Plot spatial abundance of cell types (regulatory programmes) with colour gradient
    and interpolation (from Visium anndata).
    This method supports only 7 cell types with these colours (in order, which can be changed using reorder_cmap).
    'yellow' 'orange' 'blue' 'green' 'purple' 'grey' 'white'
    :param adata: adata object with spatial coordinates in adata.obsm['spatial']
    :param color: list of adata.obs column names to be plotted
    :param kwargs: arguments to plot_spatial_general
    :return: matplotlib figure
    """

    if show_img is True:
        kwargs["show_img"] = True
        kwargs["img"] = list(adata.uns["spatial"].values())[0]["images"][img_key]

    # location coordinates
    if "spatial" in adata.uns.keys():
        kwargs["coords"] = (
            adata.obsm["spatial"] * list(adata.uns["spatial"].values())[0]["scalefactors"][f"tissue_{img_key}_scalef"]
        )
    else:
        kwargs["coords"] = adata.obsm["spatial"]

    fig = plot_spatial_general(value_df=adata.obs[color], **kwargs)  # cell abundance values

    return fig

In [ ]:
adata.obs['Ground_truth_neurons']=adata_n.obs['neurons']
adata.obs['Ground_truth_glial']=adata_g.obs['glial']

In [ ]:

clust_col = ['Ground_truth_glial']

with mpl.rc_context({'figure.figsize': (15, 15)}):
    plt.rcParams["axes.grid"] = False
    fig = plot_spatial(
        adata=adata,
        labels = clust_col,
        # labels to show on a plot
        color=clust_col,
        show_img=True,
        # 'fast' (white background) or 'dark_background'
        style='fast',
         reorder_cmap=(0,1),
        circle_diameter=10,
        img_alpha=0,
        colorbar_position='right'
    )

In [ ]:
clust_col = ['Ground_truth_neurons']

with mpl.rc_context({'figure.figsize': (15, 15)}):
    plt.rcParams["axes.grid"] = False
    fig = plot_spatial(
        adata=adata,
        labels = clust_col,
        color=clust_col,
        show_img=True,
        # 'fast' (white background) or 'dark_background'
        style='fast',
         reorder_cmap=(2,3),
        circle_diameter=10,
        img_alpha=0,
        colorbar_position='right'
    )

In [ ]:
adata_cellpie = adata.copy()

##### we run CellPie for a range of weights: [0,0.4,1,1.6,2]

In [ ]:
k=17
nmf_model = intNMF(adata_cellpie,k, epochs = 60,init = 'NNDSVD',mod1_skew=2,random_state=2)
nmf_model.fit(adata_cellpie)

In [ ]:
from itertools import combinations
from scipy import stats

for l in range(5,6):

#     adata.obs = adata.obs.drop(columns=['comb'])
#     adata_cellpie.obs = adata_cellpie.obs.drop(columns=['comb'])
    cc = list(combinations(adata_cellpie.obs.iloc[:,12:29].columns,l))

    res3=[]
    adata_new = adata_cellpie

    i=0

    for c in cc:
        adata_new.obs['comb'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    
        for j in range(0,l):
            adata_new.obs['comb'] = adata_new.obs['comb'] + adata_new.obs[cc[i][j]] 
        
        rho_n  = stats.spearmanr(adata_new.obs["comb"],adata_n.obs['neurons'])
        rho_g = stats.spearmanr(adata_new.obs["comb"],adata_g.obs['glial'])
        i=i+1
        res3.append((c,rho_n[0],rho_g[0]))

    coeff2 = pd.DataFrame(res3)
    print(l,coeff2.max())


In [ ]:
# weight 0 max for l=2
neur_max_0 =[ coeff2[0].loc[coeff2[1].idxmax()]]

In [ ]:
# weight 0 max for l=11
glial_max_0 =[ coeff2[0].loc[coeff2[2].idxmax()]]

In [ ]:
l=2
i=0
for c in neur_max_0:
    adata_cellpie.obs['CellPie_neurons_0'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_neurons_0'] = adata_cellpie.obs['CellPie_neurons_0']+adata_cellpie.obs[neur_max_0[i][j]]
    i=i+1
        

In [ ]:
l=11
i=0
for c in glial_max_0:
    adata_cellpie.obs['CellPie_glial_0'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_glial_0'] = adata_cellpie.obs['CellPie_glial_0']+adata_cellpie.obs[glial_max_0[i][j]]
    i=i+1

In [ ]:
# weight 0.4 max for l=6
neur_max_04 =[ coeff2[0].loc[coeff2[1].idxmax()]]

In [ ]:
neur_max_04

In [ ]:
# weight 0.4 max for l=5
glial_max_04 =[ coeff2[0].loc[coeff2[2].idxmax()]]

In [ ]:
l=6
i=0
for c in neur_max_04:
    adata_cellpie.obs['CellPie_neurons_0.4'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_neurons_0.4'] = adata_cellpie.obs['CellPie_neurons_0.4']+adata_cellpie.obs[neur_max_04[i][j]]
    i=i+1

In [ ]:
l=5
i=0
for c in glial_max_04:
    adata_cellpie.obs['CellPie_glial_0.4'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_glial_0.4'] = adata_cellpie.obs['CellPie_glial_0.4']+adata_cellpie.obs[glial_max_04[i][j]]
    i=i+1

In [ ]:
# weight 1 max for l=7
neur_max_1 =[ coeff2[0].loc[coeff2[1].idxmax()]]

In [ ]:
neur_max_1

In [ ]:
# weight 1 max for l=5
glial_max_1 =[ coeff2[0].loc[coeff2[2].idxmax()]]

In [ ]:
l=7
i=0
for c in neur_max_1:
    adata_cellpie.obs['CellPie_neurons'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_neurons'] = adata_cellpie.obs['CellPie_neurons']+adata_cellpie.obs[neur_max_1[i][j]]
    i=i+1

In [ ]:
l=5
i=0
for c in glial_max_1:
    adata_cellpie.obs['CellPie_glial'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_glial'] = adata_cellpie.obs['CellPie_glial']+adata_cellpie.obs[glial_max_1[i][j]]
    i=i+1

In [ ]:
# weight 1.6 max for l=3
neur_max_16 =[ coeff2[0].loc[coeff2[1].idxmax()]]

In [ ]:
l=3
i=0
for c in neur_max_16:
    adata_cellpie.obs['CellPie_neurons_16'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_neurons_16'] = adata_cellpie.obs['CellPie_neurons_16']+adata_cellpie.obs[neur_max_16[i][j]]
    i=i+1

In [ ]:
# weight 1.6 max for l=5
glial_max_16 =[ coeff2[0].loc[coeff2[2].idxmax()]]

In [ ]:
l=5
i=0
for c in glial_max_16:
    adata_cellpie.obs['CellPie_glial_16'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_glial_16'] = adata_cellpie.obs['CellPie_glial_16']+adata_cellpie.obs[glial_max_16[i][j]]
    i=i+1

In [ ]:
# weight 2 max for l=1
neur_max_2 =[ coeff2[0].loc[coeff2[1].idxmax()]]

In [ ]:
l=1
i=0
for c in neur_max_2:
    adata_cellpie.obs['CellPie_neurons_2'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_neurons_2'] = adata_cellpie.obs['CellPie_neurons_2']+adata_cellpie.obs[neur_max_2[i][j]]
    i=i+1

In [ ]:
# weight 2 max for l=5
glial_max_2 =[ coeff2[0].loc[coeff2[2].idxmax()]]

In [ ]:
l=5
i=0
for c in glial_max_2:
    adata_cellpie.obs['CellPie_glial_2'] = np.zeros(len(adata_cellpie.obs['deconv_0']))
    for j in range(0,l):
        adata_cellpie.obs['CellPie_glial_2'] = adata_cellpie.obs['CellPie_glial_2']+adata_cellpie.obs[glial_max_2[i][j]]
    i=i+1

In [ ]:
clust_col = ['CellPie_neurons']
with mpl.rc_context({'figure.figsize': (15, 15)}):
    plt.rcParams["axes.grid"] = False
    fig = plot_spatial(
        adata=adata_cellpie,
        labels = clust_col,
        color=clust_col,
        show_img=True,
        # 'fast' (white background) or 'dark_background'
        style='fast',
         reorder_cmap=(2,3),
        circle_diameter=10,
        img_alpha=0,
        colorbar_position='right'
    )

In [ ]:
clust_col = ['CellPie_glial']

with mpl.rc_context({'figure.figsize': (15, 15)}):
    plt.rcParams["axes.grid"] = False
    fig = plot_spatial(
        adata=adata_cellpie,
        labels = clust_col,
        color=clust_col,
        show_img=True,
        # 'fast' (white background) or 'dark_background'
        style='fast',
         reorder_cmap=(0,1),
        circle_diameter=10,
        img_alpha=0,
        colorbar_position='right'
    )

## stDeconvolve

In [ ]:
stdec = pd.read_csv('Data/deconprop-mouse_brain_if_17_new.csv')

In [ ]:
adata=adata[adata.obs_names.isin(stdec['Unnamed: 0']),:]

In [ ]:
# adata.X.A = adata_n.X.A

In [ ]:
adata_n=adata_n[adata_n.obs_names.isin(stdec['Unnamed: 0']),:]
adata_g=adata_g[adata_g.obs_names.isin(stdec['Unnamed: 0']),:]

In [ ]:
adata_stdec = adata.copy()

In [ ]:
for i in range(1,18):
    adata_stdec.obs[f"st_dec{i}"] = stdec.iloc[:,i].values

In [ ]:
adata_stdec.obs['comb_n'] = adata_stdec.obs['st_dec1']+adata_stdec.obs['st_dec2']+adata_stdec.obs['st_dec3']+adata_stdec.obs['st_dec5']+adata_stdec.obs['st_dec6']+adata_stdec.obs['st_dec7']+adata_stdec.obs['st_dec8'] +adata_stdec.obs['st_dec9']+adata_stdec.obs['st_dec10'] +adata_stdec.obs['st_dec13']+adata_stdec.obs['st_dec14'] +adata_stdec.obs['st_dec15']
adata_stdec.obs['comb_g'] = adata_stdec.obs['st_dec1']+adata_stdec.obs['st_dec2']+adata_stdec.obs['st_dec4']+adata_stdec.obs['st_dec6']+adata_stdec.obs['st_dec8']+adata_stdec.obs['st_dec10']+adata_stdec.obs['st_dec11'] + adata_stdec.obs['st_dec12']+adata_stdec.obs['st_dec13'] + adata_stdec.obs['st_dec14']+adata_stdec.obs['st_dec15']  +adata_stdec.obs['st_dec16']

In [ ]:
from itertools import combinations

for l in range(13,14):

    cc = list(combinations(adata_stdec.obs.iloc[:,47:64].columns,l))

    res3=[]
    i=0
    for c in cc:
        adata_stdec.obs['comb'] = np.zeros(len(adata_stdec.obs['st_dec1']))
        for j in range(0,l):
            adata_stdec.obs['comb'] = adata_stdec.obs['comb'] + adata_stdec.obs[cc[i][j]] 

        rho_n  = stats.spearmanr(adata_stdec.obs["comb"],adata_n.obs['neurons'])
        rho_g = stats.spearmanr(adata_stdec.obs["comb"],adata_g.obs['glial'])
        i=i+1
        res3.append((c,rho_n[0],rho_g[0]))
    coeff2 = pd.DataFrame(res3)
    print(l,coeff2.max())

In [ ]:
adata.obs['StDeconvolve_glial'] = adata_stdec.obs['comb_g']
adata.obs['StDeconvolve_neurons'] = adata_stdec.obs['comb_n']

## Cell2Location

In [ ]:
c2l = pd.read_csv('Data/c2l_mouse_brain_if_deconv.csv')

In [ ]:
adata_cellpie.obs['C2l_glial'] = c2l['c2l_glial'].values
adata_cellpie.obs['C2l_neurons'] = c2l['c2l_neurons'].values

In [ ]:
adata_n_c2l = adata_cellpie.copy()
adata_g_c2l = adata_cellpie.copy()

In [ ]:
sq.im.calculate_image_features(
    adata_g_c2l,
    img_g,
    features="summary",
    mask_circle=True,
    show_progress_bar=True,
    scale=1,
    key_added="features"
)

In [ ]:
sq.im.calculate_image_features(
    adata_n_c2l,
    img_n,
    features="summary",
    mask_circle=True,
    show_progress_bar=True,
    scale=1,
    key_added="features"
)

In [ ]:
adata_g_c2l.obs['glial'] = adata_g_c2l.obsm['features']['summary_ch-0_mean']
adata_n_c2l.obs['neurons'] = adata_n_c2l.obsm['features']['summary_ch-0_mean']

In [ ]:
from scipy import stats
rho, pval = stats.spearmanr(adata_cellpie.obs['C2l_neurons'],adata_n_c2l.obs['neurons'])
print('rho jointNMF:',rho)

In [ ]:
from scipy import stats
rho, pval = stats.spearmanr(adata_cellpie.obs['C2l_glial'],adata_g_c2l.obs['glial'])
print('rho jointNMF:',rho)

## Stereorscope

In [ ]:
stereo = pd.read_csv('Data/mouse_brain_stereoscope.csv')

In [ ]:
adata_cellpie.obs['Stereoscope_glial'] = stereo['Glial_stereoscope'].values
adata_cellpie.obs['Stereoscope_neurons'] = stereo['Neurons_stereoscope'].values

In [ ]:
data_glial_2=pd.concat([adata.obs['StDeconvolve_glial'],adata_cellpie.obs['CellPie_glial_2'],adata_cellpie.obs['Stereoscope_glial'],adata_cellpie.obs['C2l_glial'],adata_g_c2l.obs['glial']],axis=1)
data_glial_16=pd.concat([adata.obs['StDeconvolve_glial'],adata_cellpie.obs['CellPie_glial_16'],adata_cellpie.obs['Stereoscope_glial'],adata_cellpie.obs['C2l_glial'],adata_g_c2l.obs['glial']],axis=1)
data_glial_1=pd.concat([adata.obs['StDeconvolve_glial'],adata_cellpie.obs['CellPie_glial'],adata_cellpie.obs['Stereoscope_glial'],adata_cellpie.obs['C2l_glial'],adata_g_c2l.obs['glial']],axis=1)         
data_glial_04=pd.concat([adata.obs['StDeconvolve_glial'],adata_cellpie.obs['CellPie_glial_0.4'],adata_cellpie.obs['Stereoscope_glial'],adata_cellpie.obs['C2l_glial'],adata_g_c2l.obs['glial']],axis=1)
data_glial_0=pd.concat([adata.obs['StDeconvolve_glial'],adata_cellpie.obs['CellPie_glial_0'],adata_cellpie.obs['Stereoscope_glial'],adata_cellpie.obs['C2l_glial'],adata_g_c2l.obs['glial']],axis=1)
            

In [ ]:
data_neurons_2=pd.concat([adata.obs['StDeconvolve_neurons'],adata_cellpie.obs['CellPie_neurons_2'],adata_cellpie.obs['Stereoscope_neurons'],adata_cellpie.obs['C2l_neurons'],adata_n_c2l.obs['neurons']],axis=1)
data_neurons_16=pd.concat([adata.obs['StDeconvolve_neurons'],adata_cellpie.obs['CellPie_neurons_16'],adata_cellpie.obs['Stereoscope_neurons'],adata_cellpie.obs['C2l_neurons'],adata_n_c2l.obs['neurons']],axis=1)            
data_neurons_1=pd.concat([adata.obs['StDeconvolve_neurons'],adata_cellpie.obs['CellPie_neurons'],adata_cellpie.obs['Stereoscope_neurons'],adata_cellpie.obs['C2l_neurons'],adata_n_c2l.obs['neurons']],axis=1)
data_neurons_04=pd.concat([adata.obs['StDeconvolve_neurons'],adata_cellpie.obs['CellPie_neurons_0.4'],adata_cellpie.obs['Stereoscope_neurons'],adata_cellpie.obs['C2l_neurons'],adata_n_c2l.obs['neurons']],axis=1)
data_neurons_0=pd.concat([adata.obs['StDeconvolve_neurons'],adata_cellpie.obs['CellPie_neurons_0'],adata_cellpie.obs['Stereoscope_neurons'],adata_cellpie.obs['C2l_neurons'],adata_n_c2l.obs['neurons']],axis=1)
            

In [ ]:
plt.figure(figsize=(2, 2))
corr_2 = data_glial_2.corr(method='spearman')[['glial']]
corr_16 = data_glial_16.corr(method='spearman')[['glial']]
corr_1 = data_glial_1.corr(method='spearman')[['glial']]
corr_04 = data_glial_04.corr(method='spearman')[['glial']]
corr_0 = data_glial_0.corr(method='spearman')[['glial']]

f,(ax1,ax2,ax3,ax4,ax5, axcb) = plt.subplots(1,6, 
            gridspec_kw={'width_ratios':[4,4,4,4,4,0.4]},figsize=(8, 7))
ax1.get_shared_y_axes().join(ax2,ax3,ax4,ax5)
g1 = sns.heatmap(corr_0,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},xticklabels=False,cbar=False,ax=ax1)
g1.set_ylabel('')
g2 = sns.heatmap(corr_04,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},xticklabels=False,cbar=False,ax=ax2)
g2.set_ylabel('')
g2.set_yticks([])
g3 = sns.heatmap(corr_1,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},cbar=False, ax=ax3)
g3.set_ylabel('')
g3.set_yticks([])
g4 = sns.heatmap(corr_16,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},cbar=False,xticklabels=False, ax=ax4)
g4.set_ylabel('')
g4.set_yticks([])
g6 = sns.heatmap(corr_2,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},ax=ax5,xticklabels=False, cbar_ax=axcb)
g6.set_ylabel('')
g6.set_xlabel('')
g6.set_yticks([])
plt.gcf().subplots_adjust(bottom=0.15)
plt.savefig('glial_heatmap.jpg',dpi=500,bbox_inches='tight')

In [ ]:
plt.figure(figsize=(2, 2))
corr_2 = data_neurons_2.corr(method='spearman')[['neurons']]
corr_16 = data_neurons_16.corr(method='spearman')[['neurons']]
corr_1 = data_neurons_1.corr(method='spearman')[['neurons']]
corr_04 = data_neurons_04.corr(method='spearman')[['neurons']]
corr_0 = data_neurons_0.corr(method='spearman')[['neurons']]

f,(ax1,ax2,ax3,ax4,ax6, axcb) = plt.subplots(1,6, 
            gridspec_kw={'width_ratios':[4,4,4,4,4,0.8]},figsize=(8, 7))
ax1.get_shared_y_axes().join(ax2,ax3,ax4,ax5)
g1 = sns.heatmap(corr_0,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},xticklabels=False,cbar=False,ax=ax1)
g1.set_ylabel('')
g2 = sns.heatmap(corr_04,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},xticklabels=False,cbar=False,ax=ax2)
g2.set_ylabel('')
g2.set_yticks([])
g3 = sns.heatmap(corr_1,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},cbar=False, ax=ax3)
g3.set_ylabel('')
g3.set_yticks([])
g4 = sns.heatmap(corr_16,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},cbar=False,xticklabels=False, ax=ax4)
g4.set_ylabel('')
g4.set_yticks([])
g6 = sns.heatmap(corr_2,cmap=sns.color_palette("crest", as_cmap=True),vmin=0,square=True,annot=True,annot_kws={"fontsize":25},ax=ax6,xticklabels=False, cbar_ax=axcb)
g6.set_ylabel('')
g6.set_xlabel('')
g6.set_yticks([])

plt.gcf().subplots_adjust(bottom=0.15)
plt.savefig('neurons_heatmap.jpg',dpi=500,bbox_inches='tight')

In [ ]:
nmf_model = intNMF(adata_cellpie,17, epochs = 60,init = 'NNDSVD',mod1_skew=1,random_state=2)
nmf_model.fit(adata_cellpie)

In [ ]:
# get the top genes per topic
l=cp.get_genes_topic(adata_cellpie,nmf_model.phi_expr)

In [ ]:
l.T.to_csv('marker_genes_mouse_if_repr.csv')

In [ ]:
genes=l.T

In [ ]:
adata_l=sc.AnnData(l)

In [ ]:
adata_l.write('mouse_if_genes_repr.h5ad')